In [96]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import scipy.io
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
import sklearn.datasets
import re

from tensorflow.python.framework import ops
%matplotlib inline

## Data Preprocessing

In [2]:
rawdata = pd.read_excel('SBA_Loan_data_.xlsx')

In [3]:
rawdata.head()

,Program,BorrName,BorrStreet,BorrCity,BorrState,BorrZip,CDC_Name,CDC_Street,CDC_City,CDC_State,...,InitialInterestRate,TermInMonths,NaicsCode,NaicsDescription,ProjectCounty,ProjectState,BusinessType,LoanStatus,ChargeOffDate,GrossChargeOffAmount
0,504,ROBERT G. FIELDS AND MARY D. F,55 & RICHLAND,KANSAS CITY,KS,66106,Avenue Area Incorporated,3324 Emerald Lane,Jefferson City,MO,...,NaN,12,NaN,NaN,WYANDOTTE,KS,INDIVIDUAL,PIF,NaT,0.0
1,504,CANDLELAMP COMPANY,3454 NIKI WAY,RIVERSIDE,CA,92507,CDC Small Business Finance Cor,2448 Historic Decatur,San Diego,CA,...,NaN,240,NaN,NaN,RIVERSIDE,CA,INDIVIDUAL,PIF,NaT,0.0
2,504,"NATIONAL COMPOSITES, INC.",2303 N. BENDIX DRIVE,SOUTH BEND,IN,46628,Business Development Corporati,218 W. Washington Street,South Bend,IN,...,NaN,120,NaN,NaN,ST JOSEPH,IN,CORPORATION,PIF,NaT,0.0
3,504,WENDY'S OF HARRISBURG,OUTLOT A. TR. 45N & SMALL ST,HARRISBURG,IL,62946,Small Business Growth Corporat,2401 West White Oaks Drive,Springfield,IL,...,NaN,240,NaN,NaN,SALINE,IL,CORPORATION,CHGOFF,2003-03-28,0.0
4,504,"RON SAUNORIS GARDEN CTR, INC.",13747 W. 159TH ST.,LOCKPORT,IL,60441,MISSNG/INACTV LENDER,NaN,NaN,NaN,...,NaN,240,NaN,NaN,WILL,IL,CORPORATION,CANCLD,NaT,0.0


In [4]:
rawdata.columns

Index(['Program', 'BorrName', 'BorrStreet', 'BorrCity', 'BorrState', 'BorrZip',
       'CDC_Name', 'CDC_Street', 'CDC_City', 'CDC_State', 'CDC_Zip',
       'ThirdPartyLender_Name', 'ThirdPartyLender_City',
       'ThirdPartyLender_State', 'ThirdPartyDollars', 'GrossApproval',
       'ApprovalDate', 'ApprovalFiscalYear', 'DeliveryMethod', 'subpgmdesc',
       'InitialInterestRate', 'TermInMonths', 'NaicsCode', 'NaicsDescription',
       'ProjectCounty', 'ProjectState', 'BusinessType', 'LoanStatus',
       'ChargeOffDate', 'GrossChargeOffAmount'],
      dtype='object')

In [5]:
# drop the first column
print(np.unique(rawdata['Program']))
#checkCol(rawdata, 'Program', numRow, isCata = False)
rawdata = rawdata.drop(['Program'],axis = 1)

[504]


In [6]:
# remove cancelled and exempt
rawdata = rawdata[(rawdata['LoanStatus'] == 'PIF') | (rawdata['LoanStatus'] == 'CHGOFF')]
rawdata['target'] = [int(i) for i in rawdata['LoanStatus'] == 'PIF']
rawdata.head()

,BorrName,BorrStreet,BorrCity,BorrState,BorrZip,CDC_Name,CDC_Street,CDC_City,CDC_State,CDC_Zip,...,TermInMonths,NaicsCode,NaicsDescription,ProjectCounty,ProjectState,BusinessType,LoanStatus,ChargeOffDate,GrossChargeOffAmount,target
0,ROBERT G. FIELDS AND MARY D. F,55 & RICHLAND,KANSAS CITY,KS,66106,Avenue Area Incorporated,3324 Emerald Lane,Jefferson City,MO,65109.0,...,12,NaN,NaN,WYANDOTTE,KS,INDIVIDUAL,PIF,NaT,0.0,1
1,CANDLELAMP COMPANY,3454 NIKI WAY,RIVERSIDE,CA,92507,CDC Small Business Finance Cor,2448 Historic Decatur,San Diego,CA,92106.0,...,240,NaN,NaN,RIVERSIDE,CA,INDIVIDUAL,PIF,NaT,0.0,1
2,"NATIONAL COMPOSITES, INC.",2303 N. BENDIX DRIVE,SOUTH BEND,IN,46628,Business Development Corporati,218 W. Washington Street,South Bend,IN,46601.0,...,120,NaN,NaN,ST JOSEPH,IN,CORPORATION,PIF,NaT,0.0,1
3,WENDY'S OF HARRISBURG,OUTLOT A. TR. 45N & SMALL ST,HARRISBURG,IL,62946,Small Business Growth Corporat,2401 West White Oaks Drive,Springfield,IL,62704.0,...,240,NaN,NaN,SALINE,IL,CORPORATION,CHGOFF,2003-03-28,0.0,0
6,"ROCKY MTN EMPLOYE BENEFTS, INC",3200 S. 700 E.,SALT LAKE CITY,UT,84106,Mountain West Small Business F,2595 East 3300 South,Salt Lake City,UT,84109.0,...,240,NaN,NaN,SALT LAKE,UT,CORPORATION,PIF,NaT,0.0,1


In [85]:
# add feature GSP in the borrower state and year
state = pd.read_csv('state.csv',header = None)
state.set_index(0,inplace = True)
statedict = state.to_dict()[1]
statedict['District of Columbia'] = 'DC'
statedict['Puerto Rico'] = 'PR'
statedict['Guam'] = 'GU'
gsp = pd.read_excel('GSP.xls')
gsp['state'] = gsp['GeoName'].map(lambda x: statedict[x] if x in statedict else '')
gsp.set_index('state',inplace = True)
gsp_pr = pd.read_excel('GSP-PR.xls')
gsp_pr['ind'] = gsp_pr['year'].map(lambda x: str(x)[:4])
gsp_pr.set_index('ind',inplace = True)

In [89]:
Borrgsp = []
BorrState = list(rawdata['BorrState'])
year = list(rawdata['ApprovalFiscalYear'])
for i in range(len(year)):
    if BorrState[i] == 'PR':
        Borrgsp.append(gsp_pr.loc[str(year[i]),'GSP'])
    else:
        Borrgsp.append(gsp.loc[BorrState[i],str(year[i])] if BorrState[i] in gsp.index else 0)
rawdata['Borrgsp'] = Borrgsp

In [91]:
# add feature unemployment rate in the borrower state and year
import re
unemploydict = {}
with open('unemply_dict.txt') as file:
    lines = file.readlines()

for i in range(len(lines)):
    if i%2 == 0:
        line = lines[i]
        state = re.search('(wide )(.+)(Season)',line).group(2)
        state = state[:(len(state)//2)][:-1]
        unemploydict[re.search('LASST(\w+)',line).group(0)] = state
print(unemploydict)       

{'LASST010000000000003': 'Alabama', 'LASST020000000000003': 'Alaska', 'LASST040000000000003': 'Arizona', 'LASST050000000000003': 'Arkansas', 'LASST060000000000003': 'California', 'LASST080000000000003': 'Colorado', 'LASST090000000000003': 'Connecticut', 'LASST100000000000003': 'Delaware', 'LASST110000000000003': 'District of Columbia', 'LASST120000000000003': 'Florida', 'LASST130000000000003': 'Georgia', 'LASST150000000000003': 'Hawaii', 'LASST160000000000003': 'Idaho', 'LASST170000000000003': 'Illinois', 'LASST180000000000003': 'Indiana', 'LASST190000000000003': 'Iowa', 'LASST200000000000003': 'Kansas', 'LASST210000000000003': 'Kentucky', 'LASST220000000000003': 'Louisiana', 'LASST230000000000003': 'Maine', 'LASST240000000000003': 'Maryland', 'LASST250000000000003': 'Massachusetts', 'LASST260000000000003': 'Michigan', 'LASST270000000000003': 'Minnesota', 'LASST280000000000003': 'Mississippi', 'LASST290000000000003': 'Missouri', 'LASST300000000000003': 'Montana', 'LASST310000000000003'

In [92]:
unemply_all = pd.read_excel('unemploy_all.xlsx')
flag = unemply_all['Series ID'].map(lambda x: x in unemploydict)
unemply_all['flag'] = flag
unemply_all = unemply_all[unemply_all['flag'] == True]
unemply_all['state'] = unemply_all['Series ID'].map(lambda x: statedict[unemploydict[x]])
unemply_all.set_index('state',inplace = True)
#unemply_all.head()
month = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
for y in range(1990,2014):
    cols = [m+'\n'+str(y) for m in month]
    unemply_all[y] = list(unemply_all.loc[:,cols].mean(axis=1))

In [93]:
unemply_all.head()

,Series ID,Jan 1990,Feb 1990,Mar 1990,Apr 1990,May 1990,Jun 1990,Jul 1990,Aug 1990,Sep 1990,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
state,,,,,,,,,,,,,,,,,,,,,
AL,LASST010000000000003,6.7,6.7,6.7,6.7,6.6,6.6,6.7,6.7,6.8,...,5.700000,4.500000,4.066667,3.975,5.716667,10.991667,10.541667,9.616667,7.983333,7.216667
AK,LASST020000000000003,7.2,7.2,7.2,7.2,7.2,7.2,7.2,7.2,7.2,...,7.450000,6.883333,6.625000,6.350,6.708333,7.733333,7.875000,7.566667,7.133333,7.000000
AZ,LASST040000000000003,5.2,5.3,5.4,5.4,5.3,5.3,5.3,5.2,5.2,...,5.041667,4.666667,4.225000,3.825,6.141667,9.925000,10.350000,9.500000,8.333333,7.741667
AR,LASST050000000000003,6.7,6.7,6.7,6.7,6.7,6.8,6.9,6.9,7.0,...,5.700000,5.225000,5.175000,5.300,5.500000,7.825000,8.191667,8.291667,7.575000,7.216667
CA,LASST060000000000003,5.2,5.2,5.2,5.3,5.4,5.5,5.7,5.9,6.1,...,6.208333,5.400000,4.916667,5.375,7.291667,11.141667,12.225000,11.708333,10.375000,8.941667


In [94]:
unemploy = []
for i in range(len(year)):
    unemploy.append(unemply_all.loc[BorrState[i],year[i]] if BorrState[i] in unemply_all.index else 0)
rawdata['unemploy'] = unemploy

In [95]:
rawdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54806 entries, 0 to 140496
Data columns (total 32 columns):
BorrName                  54806 non-null object
BorrStreet                54798 non-null object
BorrCity                  54806 non-null object
BorrState                 54806 non-null object
BorrZip                   54806 non-null int64
CDC_Name                  54806 non-null object
CDC_Street                54614 non-null object
CDC_City                  54614 non-null object
CDC_State                 54614 non-null object
CDC_Zip                   54614 non-null float64
ThirdPartyLender_Name     16953 non-null object
ThirdPartyLender_City     16953 non-null object
ThirdPartyLender_State    17068 non-null object
ThirdPartyDollars         16881 non-null float64
GrossApproval             54806 non-null int64
ApprovalDate              54806 non-null datetime64[ns]
ApprovalFiscalYear        54806 non-null int64
DeliveryMethod            54806 non-null object
subpgmdesc         

In [128]:
numRow = len(rawdata)
def checkCol(df, colName, numRow, isCata = True):
    print(colName, ':')
    print('Any NA?: ', df[colName].isnull().values.any())
    if df[colName].isnull().values.any():
        numNA = sum(df[colName].isnull().values)
        print ('    # of NA: ', numNA)
        print ('    NA%:     ', numNA/numRow)
    if isCata:
        levelList = df[colName].unique()
        print('Different levels: ', len(levelList), levelList)
        fillna(df, colName, isCata = True)
        
    else:
        print('range: ', min(df[colName].astype(float).dropna()), max(df[colName].astype(float).dropna()))
        fillna(df, colName, isCata = False)
        
def fillna(df, colName, isCata = True):
    if isCata:
        df[colName] = df[colName].fillna(0)
    else:
        df[colName] = df[colName].fillna('0')
    return df


In [147]:
catCols = ['BorrName', 'BorrStreet', 'BorrCity', 'BorrState','CDC_Name', 'CDC_Street', 'CDC_City', 'CDC_State', 'ThirdPartyLender_Name', 'ThirdPartyLender_City',
       'ThirdPartyLender_State','ApprovalDate', 'ApprovalFiscalYear', 'DeliveryMethod', 'subpgmdesc','NaicsCode', 'NaicsDescription',
       'ProjectCounty', 'ProjectState', 'BusinessType', 
       ]
numCols = ['BorrZip','CDC_Zip','ThirdPartyDollars', 'GrossApproval','InitialInterestRate', 'TermInMonths','Borrgsp','unemploy']
for col in catCols:
    checkCol(rawdata, col, numRow, isCata = True)
for col in numCols:
    checkCol(rawdata, col, numRow, isCata = False)

BorrName :
Any NA?:  False
Different levels:  52068 [7999, 31530, 49329, 24493, 8290, ..., 23615, 21699, 48909, 4972, 48911]
Length: 52068
Categories (52068, int64): [7999, 31530, 49329, 24493, ..., 21699, 48909, 4972, 48911]
BorrStreet :
Any NA?:  False
Different levels:  53510 [12344, 74, 48400, 32805, 51513, ..., 39382, 44618, 36154, 21430, 21215]
Length: 53510
Categories (53510, int64): [12344, 74, 48400, 32805, ..., 44618, 36154, 21430, 21215]
BorrCity :
Any NA?:  False
Different levels:  7442 [653, 3861, 4622, 7408, 4129, ..., 5119, 6955, 5155, 5698, 4051]
Length: 7442
Categories (7442, int64): [653, 3861, 4622, 7408, ..., 6955, 5155, 5698, 4051]
BorrState :
Any NA?:  False
Different levels:  54 [49, 21, 48, 47, 30, ..., 39, 16, 13, 42, 32]
Length: 54
Categories (54, int64): [49, 21, 48, 47, ..., 16, 13, 42, 32]
CDC_Name :
Any NA?:  False
Different levels:  296 [230, 51, 43, 266, 206, ..., 256, 279, 9, 130, 31]
Length: 296
Categories (296, int64): [230, 51, 43, 266, ..., 279, 9, 

In [148]:
# transform data type
for col in catCols:
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(rawdata[col].values.astype('str')))
    rawdata[col] = lbl.transform(list(rawdata[col].values.astype('str')))
    rawdata[col] = rawdata[col].astype('category')
rawdata.dtypes

BorrName                  category
BorrStreet                category
BorrCity                  category
BorrState                 category
BorrZip                      int64
CDC_Name                  category
CDC_Street                category
CDC_City                  category
CDC_State                 category
CDC_Zip                     object
ThirdPartyLender_Name     category
ThirdPartyLender_City     category
ThirdPartyLender_State    category
ThirdPartyDollars           object
GrossApproval                int64
ApprovalDate              category
ApprovalFiscalYear        category
DeliveryMethod            category
subpgmdesc                category
InitialInterestRate         object
TermInMonths                 int64
NaicsCode                 category
NaicsDescription          category
ProjectCounty             category
ProjectState              category
BusinessType              category
LoanStatus                category
ChargeOffDate             category
GrossChargeOffAmount

In [149]:
# split the data
y = list(rawdata['target'])
X = rawdata[numCols+catCols]
train_X, test_X, train_y, test_y = train_test_split(X,y,test_size=0.2,random_state=1)
train_X, val_X, train_y, val_y = train_test_split(train_X,train_y,test_size=0.2,random_state=1)


## Baseline: Logistic Regression

In [150]:
# standardization and one-hot encoding
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numCols),
                                               ('cat', categorical_transformer, catCols)])
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression(solver='lbfgs'))])

clf.fit(train_X, train_y)
print("model score: %.3f" % clf.score(val_X, val_y))
print(classification_report(val_y, clf.predict(val_X)))
print(roc_auc_score(val_y, clf.predict_proba(val_X)[:,1]))

C:\Users\THINKPAD\AppData\Roaming\Python\Python36\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\THINKPAD\AppData\Roaming\Python\Python36\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\THINKPAD\AppData\Roaming\Python\Python36\site-packages\sklearn\linear_model\logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\THINKPAD\AppData\Roaming\Python\Python36\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
C:\Users\THINKPAD\AppData\Roaming\Pytho

model score: 0.867
              precision    recall  f1-score   support

           0       0.63      0.41      0.50      1408
           1       0.89      0.95      0.92      7361

   micro avg       0.87      0.87      0.87      8769
   macro avg       0.76      0.68      0.71      8769
weighted avg       0.85      0.87      0.86      8769

0.8411669957453904


C:\Users\THINKPAD\AppData\Roaming\Python\Python36\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
